# Introduction

This notebook explores the characteristics of inactive users. Inactive users are defined as users that create an account but don't complete any modules.

# Imports and API Keys

You will need your own username and password to the SQL database to run the code

In [2]:
import os
import pandas as pd
import mysql.connector
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [3]:
import chart_studio

chart_studio.tools.set_credentials_file(username = os.environ.get('cs_user'), api_key = os.environ.get('cs_key'))

import chart_studio.plotly as py
import chart_studio.tools as tls

In [4]:
usr = os.environ.get('sql_user')
pwd = os.environ.get('sql_pwd')
host = os.environ.get('sql_host')
db = os.environ.get('sql_db')

def query(sql):
    
    cnx = mysql.connector.connect(user=usr, 
                              password=pwd,
                              host=host,
                              database=db,
                                 port=3306)
    
    df = pd.read_sql(sql, cnx)
    cnx.close()
    return df

# Queries

Note that users that completed no modules at all are filtered out

Query for inactive users (people that only complete one module), number of modules they completed (1), whether they're referred in or not, their institution, and age:

In [ ]:
inactive_query = '''
SELECT tab1.id, num_modules, survey_answer,
CASE
	WHEN tab1.id IN 
    (
    SELECT referred_id
	FROM referral_relationship
    ) THEN 'yes'
    ELSE 'no'
END AS referred,
CASE
	WHEN institution IS NULL THEN 'null'
    ELSE institution
END as institution,
CASE
	WHEN age IS NULL THEN -1
    ELSE age
END as age
FROM
(
SELECT 
    u.id, 
    count(DISTINCT ecm.education_module_id) as num_modules, 
    i.code as institution, FLOOR((CURDATE() - u.birthday) / 10000) AS age
FROM user u
LEFT JOIN education_completed_module ecm
ON ecm.user_id = u.id
LEFT JOIN institution i
ON i.id = u.institution_id
WHERE u.date_created IS NOT NULL
GROUP BY u.id
) AS tab1
LEFT JOIN 
(
SELECT uf.user_id AS id,
CASE
	WHEN aer.selected_answer_index = 0 THEN 'Gift Card Rewards'
    WHEN aer.selected_answer_index = 1 THEN 'Financial Literacy'
    WHEN aer.selected_answer_index = 2 THEN 'Daily News Updates'
END AS survey_answer
FROM activity_card ac
LEFT JOIN activity_education_response aer
ON ac.id = aer.activity_card_id
LEFT JOIN user_family uf
ON uf.family_id = ac.user_family_id
WHERE source_id BETWEEN 94860 AND 94862
) AS survey_tab
ON survey_tab.id = tab1.id
WHERE num_modules = 1
ORDER BY id
'''

inactive_df = query(inactive_query)
inactive_df

Query for active users, number of modules they completed (>0), whether they're referred in or not, their institution, and age:

In [17]:
active_query = '''
SELECT id, num_modules,
CASE
	WHEN answer = 0 THEN 'Gift Card Rewards'
    WHEN answer = 1 THEN 'Financial Literacy'
    WHEN answer = 2 THEN 'Daily News Updates'
    ELSE 'null'
END AS survey_answer,
CASE
	WHEN id IN 
    (
    SELECT referred_id
	FROM referral_relationship
    )
    THEN 'yes'
    ELSE 'no'
END AS referred,
CASE
	WHEN institution IS NULL THEN 'null'
    ELSE institution
END as institution,
CASE
	WHEN age IS NULL THEN -1
    ELSE age
END as age
FROM
(
SELECT id, num_modules, institution, age
FROM
(
SELECT 
    u.id, 
    count(DISTINCT ecm.education_module_id) as num_modules, 
    i.code as institution, FLOOR((CURDATE() - u.birthday) / 10000) AS age, 
    answer
FROM user u
LEFT JOIN education_completed_module ecm
ON ecm.user_id = u.id
LEFT JOIN institution i
ON i.id = u.institution_id
LEFT JOIN 
(
SELECT uf.user_id AS id, aer.selected_answer_index AS answer
FROM activity_card ac
LEFT JOIN activity_education_response aer
ON ac.id = aer.activity_card_id
LEFT JOIN user_family uf
ON uf.family_id = ac.user_family_id
WHERE source_id BETWEEN 94860 AND 94862
) AS survey_table
ON survey_table.id = u.id
WHERE u.date_created IS NOT NULL
GROUP BY u.id
) AS tab1
WHERE num_modules > 0
) mod_table
WHERE num_modules > 1
ORDER BY id
'''

active_df = query(active_query)
active_df

,id,num_modules,referred,institution,age
0,130,4,no,REVOLUTION,32
1,131,43,no,DEMODAY,15
2,132,5,no,DEMODAY,13
3,135,9,no,DEMODAY,13
4,136,23,no,DEMODAY,19
...,...,...,...,...,...
32267,58234,5,no,ZOGO123,30
32268,58237,2,no,ZOGO123,15
32269,58239,4,no,ZOGO123,47
32270,58240,4,no,DIAMONDCU,18


Query for inactive users, including survey answer, only for those that answered the survey:

In [8]:
inactive_survey_query = '''
SELECT id, num_modules, institution, age, survey_answer,
CASE
	WHEN id IN 
    (
    SELECT referred_id
	FROM referral_relationship
    ) THEN 'yes'
    ELSE 'no'
END AS referred
FROM
(
SELECT 
    u.id, 
    count(DISTINCT ecm.education_module_id) as num_modules, 
    i.code as institution, FLOOR((CURDATE() - u.birthday) / 10000) AS age,
    survey_answer
FROM user u
LEFT JOIN education_completed_module ecm
ON ecm.user_id = u.id
LEFT JOIN institution i
ON i.id = u.institution_id
RIGHT JOIN
(
SELECT uf.user_id AS id,
CASE
	WHEN aer.selected_answer_index = 0 THEN 'Gift Card Rewards'
    WHEN aer.selected_answer_index = 1 THEN 'Financial Literacy'
    WHEN aer.selected_answer_index = 2 THEN 'Daily News Updates'
END AS survey_answer
FROM activity_card ac
LEFT JOIN activity_education_response aer
ON ac.id = aer.activity_card_id
LEFT JOIN user_family uf
ON uf.family_id = ac.user_family_id
WHERE source_id BETWEEN 94860 AND 94862
) survey_tab
ON u.id = survey_tab.id
WHERE u.date_created IS NOT NULL
GROUP BY u.id
) full_tab
WHERE num_modules = 1
'''

inactive_survey = query(inactive_survey_query)
inactive_survey

,id,num_modules,institution,age,survey_answer,referred
0,56017,1,ZOGO123,26.0,Financial Literacy,no
1,53896,1,ZOGO123,27.0,Gift Card Rewards,yes
2,19177,1,ZOGO123,37.0,Daily News Updates,yes
3,52891,1,ZOGO123,14.0,Financial Literacy,no
4,20369,1,ZOGO123,NaN,Financial Literacy,yes
...,...,...,...,...,...,...
541,48063,1,ZOGO123,22.0,Gift Card Rewards,no
542,53717,1,ZOGO123,34.0,Financial Literacy,no
543,28452,1,DIAMONDCU,34.0,Gift Card Rewards,no
544,16115,1,ZOGO123,45.0,Gift Card Rewards,yes


Query for active users, including survey answer, only for those that answered the survey:

In [9]:
active_survey_query = '''
SELECT id, num_modules, institution, age, survey_answer,
CASE
	WHEN id IN 
    (
    SELECT referred_id
	FROM referral_relationship
    ) THEN 'yes'
    ELSE 'no'
END AS referred
FROM
(
SELECT 
    u.id, 
    count(DISTINCT ecm.education_module_id) as num_modules, 
    i.code as institution, FLOOR((CURDATE() - u.birthday) / 10000) AS age,
    survey_answer
FROM user u
LEFT JOIN education_completed_module ecm
ON ecm.user_id = u.id
LEFT JOIN institution i
ON i.id = u.institution_id
RIGHT JOIN
(
SELECT uf.user_id AS id,
CASE
	WHEN aer.selected_answer_index = 0 THEN 'Gift Card Rewards'
    WHEN aer.selected_answer_index = 1 THEN 'Financial Literacy'
    WHEN aer.selected_answer_index = 2 THEN 'Daily News Updates'
END AS survey_answer
FROM activity_card ac
LEFT JOIN activity_education_response aer
ON ac.id = aer.activity_card_id
LEFT JOIN user_family uf
ON uf.family_id = ac.user_family_id
WHERE source_id BETWEEN 94860 AND 94862
) survey_tab
ON u.id = survey_tab.id
WHERE u.date_created IS NOT NULL
GROUP BY u.id
) full_tab
WHERE num_modules > 1
'''

active_survey = query(active_survey_query)
active_survey

,id,num_modules,institution,age,survey_answer,referred
0,812,3,PYRAMID,NaN,Financial Literacy,no
1,858,2,ZOGO123,22.0,Gift Card Rewards,no
2,868,48,COMWIDE,54.0,Gift Card Rewards,no
3,876,6,PYRAMID,13.0,Financial Literacy,no
4,911,6,COMWIDE,42.0,Financial Literacy,yes
...,...,...,...,...,...,...
3683,59148,2,ZOGO123,35.0,Financial Literacy,no
3684,59154,2,ZOGO123,23.0,Financial Literacy,yes
3685,59165,2,ZOGO123,16.0,Gift Card Rewards,no
3686,59166,4,ZOGO123,33.0,Gift Card Rewards,no


# Data Manipulation & Visualization

## What proportion of inactive users are referred in as compared to active users?

In [18]:
inactive_ref_df = inactive_df.groupby(['referred']).count().reset_index().iloc[:, :2].rename(columns = {'id' : 'count'})
inactive_ref_df['prop'] = inactive_ref_df['count'] / inactive_ref_df['count'].sum()
inactive_ref_df

,referred,count,prop
0,no,1971,0.204567
1,yes,7664,0.795433


In [19]:
active_ref_df = active_df.groupby(['referred']).count().reset_index().iloc[:, :2].rename(columns = {'id' : 'count'})
active_ref_df['prop'] = active_ref_df['count'] / active_ref_df['count'].sum()
active_ref_df

,referred,count,prop
0,no,15609,0.48367
1,yes,16663,0.51633


In [25]:
fig_ref_comp = go.Figure(data=[
    go.Bar(name='Inactive', x=inactive_ref_df['referred'], y=inactive_ref_df['prop']),
    go.Bar(name='Active', x=active_ref_df['referred'], y=active_ref_df['prop'])
])


fig_ref_comp.update_layout(barmode='group', title = 'Inactive Users Are More Likely to Be Referred Users <br>(Users that Dont Complete Any Modules Filtered Out)',
                           xaxis_title = 'Referred', yaxis_title = 'Proportion')
fig_ref_comp.show()


## What institutions are inactive users more likely to be part of?

In [32]:
inactive_inst_df = inactive_df.groupby(['institution']).count().reset_index().iloc[:, :2].rename(columns = {'id' : 'count'})
inactive_inst_df['prop'] = inactive_inst_df['count'] / inactive_inst_df['count'].sum()
inactive_inst_df

,institution,count,prop
0,BAYPORT,88,0.009133
1,BWA,9,0.000934
2,CFCU,3,0.000311
3,CLINCHFIELD,1,0.000104
4,COMWIDE,410,0.042553
5,CSCU,16,0.001661
6,CTCU,1,0.000104
7,DEMODAY,10,0.001038
8,DIAMONDCU,311,0.032278
9,FBT,9,0.000934


In [33]:
active_inst_df = active_df.groupby(['institution']).count().reset_index().iloc[:, :2].rename(columns = {'id' : 'count'})
active_inst_df['prop'] = active_inst_df['count'] / active_inst_df['count'].sum()
active_inst_df

,institution,count,prop
0,BAYPORT,1730,0.053607
1,BWA,33,0.001023
2,CACLFCU,2,0.000062
3,CLINCHFIELD,1,0.000031
4,COMWIDE,1942,0.060176
5,CSCU,86,0.002665
6,CTCU,2,0.000062
7,DEMODAY,16,0.000496
8,DIAMONDCU,1147,0.035542
9,DISCOVER,1,0.000031


In [42]:
fig_inst_comp = go.Figure(data=[
    go.Bar(name='Inactive', 
           x=inactive_inst_df[inactive_inst_df['institution'].isin(['BAYPORT', 'COMWIDE', 'DIAMONDCU', 'ZOGO123'])]['institution'], 
           y=inactive_inst_df[inactive_inst_df['institution'].isin(['BAYPORT', 'COMWIDE', 'DIAMONDCU', 'ZOGO123'])]['prop']),
    go.Bar(name='Active', 
           x=active_inst_df[active_inst_df['institution'].isin(['BAYPORT', 'COMWIDE', 'DIAMONDCU', 'ZOGO123'])]['institution'], 
           y=active_inst_df[active_inst_df['institution'].isin(['BAYPORT', 'COMWIDE', 'DIAMONDCU', 'ZOGO123'])]['prop'])
])


fig_inst_comp.update_layout(barmode='group', 
                            title = "ZOGO123 users more likely to be inactive, very few inactive Bayport users<br>Including only major clients",
                            xaxis_title = 'Institution', yaxis_title = 'Proportion of Users')
fig_inst_comp.show()


In [35]:
inactive_inst_df[inactive_inst_df['institution'].isin(['BAYPORT', 'COMWIDE', 'DIAMONDCU', 'ZOGO123'])]

,institution,count,prop
0,BAYPORT,88,0.009133
4,COMWIDE,410,0.042553
8,DIAMONDCU,311,0.032278
33,ZOGO123,8509,0.883134


## What age group are inactive users more likely to be part of?

Note that users that didn't input their birthday are excluded.

In [28]:
bins = [-1, 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

inactive_df['binned'] = pd.cut(inactive_df['age'], bins, right = False)
active_df['binned'] = pd.cut(active_df['age'], bins, right = False)

inactive_age1 = inactive_df.groupby(['binned']).count().reset_index().iloc[:,:2].rename(columns = {'id' : 'count'}).drop(columns = ['binned']).drop([0])
active_age1 = active_df.groupby(['binned']).count().reset_index().iloc[:,:2].rename(columns = {'id' : 'count'}).drop(columns = ['binned']).drop([0])

rows = ["0~9", "10~19", "20~29", "30~39", "40~49", "50~59", "60~69", "70~79", "80~89", "90~100"]

inactive_age1['age_range'] = rows
active_age1['age_range'] = rows

inactive_age1['prop'] = inactive_age1['count'] / inactive_age1['count'].sum()
active_age1['prop'] = active_age1['count'] / active_age1['count'].sum()

In [29]:
inactive_age1

,count,age_range,prop
1,0,0~9,0.000000
2,790,10~19,0.113083
3,2368,20~29,0.338964
4,2730,30~39,0.390782
5,774,40~49,0.110793
6,251,50~59,0.035929
7,61,60~69,0.008732
8,10,70~79,0.001431
9,0,80~89,0.000000
10,2,90~100,0.000286


In [30]:
active_age1

,count,age_range,prop
1,0,0~9,0.000000
2,6808,10~19,0.213016
3,12347,20~29,0.386327
4,7909,30~39,0.247466
5,3166,40~49,0.099061
6,1269,50~59,0.039706
7,382,60~69,0.011952
8,68,70~79,0.002128
9,9,80~89,0.000282
10,2,90~100,0.000063


In [40]:
fig_age_comp = go.Figure(data=[
    go.Bar(name='Inactive', x=inactive_age1['age_range'], y=inactive_age1['prop']),
    go.Bar(name='Active', x=active_age1['age_range'], y=active_age1['prop'])
])


fig_age_comp.update_layout(barmode='group', title = '30s & 40s More Likely to Be Inactive <br>(Filtered Out Users that Dont Complete Any Modules)',
                           xaxis_title = 'Age Group', yaxis_title = 'Proportion of Users')
fig_age_comp.show()


## What did the users answer as the reason that they use Zogo?

Note that this analysis is just for users that answered the survey.

In [10]:
inactive_survey_df = inactive_survey.groupby(['survey_answer']).count().reset_index().iloc[:, :2].rename(columns = {'id' : 'count'})
inactive_survey_df['prop'] = inactive_survey_df['count'] / inactive_survey_df['count'].sum()
inactive_survey_df

,survey_answer,count,prop
0,Daily News Updates,26,0.047619
1,Financial Literacy,268,0.490842
2,Gift Card Rewards,252,0.461538


In [11]:
active_survey_df = active_survey.groupby(['survey_answer']).count().reset_index().iloc[:, :2].rename(columns = {'id' : 'count'})
active_survey_df['prop'] = active_survey_df['count'] / active_survey_df['count'].sum()
active_survey_df

,survey_answer,count,prop
0,Daily News Updates,207,0.056128
1,Financial Literacy,1889,0.512202
2,Gift Card Rewards,1592,0.431670


In [17]:
fig_survey_comp = go.Figure(data=[
    go.Bar(name='Inactive', x=inactive_survey_df['survey_answer'], y=inactive_survey_df['prop']),
    go.Bar(name='Active', x=active_survey_df['survey_answer'], y=active_survey_df['prop'])
])


fig_survey_comp.update_layout(barmode='group', 
                              title = 'Proportion of Inactive Users that think Gift Cards is Most Important Slightly Higher',
                              xaxis_title = 'Answer to which of these is most important to you when you use Zogo?',
                              yaxis_title = 'Proportion of User Group')
fig_survey_comp.show()


# Export Graphs

In [17]:
py.plot(fig_ref_comp, name = "Inactive Users Referral Comparison", auto_open = True)
py.plot(fig_inst_comp, name = "Inactive Users Institution Comparison", auto_open = True)
py.plot(fig_age_comp, name = "Inactive Users Age Comparison", auto_open = True)

'https://plotly.com/~woonggyu.jin/160/'

In [26]:
py.plot(fig_ref_comp, name = "Inactive Users Referral Comparison, Filtered Out 0 Modules", auto_open = True)

'https://plotly.com/~woonggyu.jin/164/'

In [43]:
py.plot(fig_inst_comp, name = "Inactive Users Institution Comparison, Filtered Out 0 Modules", auto_open = True)

'https://plotly.com/~woonggyu.jin/170/'

In [41]:
py.plot(fig_age_comp, name = "Inactive Users Age Comparison, Filtered Out 0 Modules", auto_open = True)

'https://plotly.com/~woonggyu.jin/168/'

In [18]:
py.plot(fig_survey_comp, name = "Inactive Users Survey Answer Comparison", auto_open = True)

'https://plotly.com/~woonggyu.jin/172/'